DMD FUNCTIONS - FAT matrix X - for ONLINE DMD comparison - Zhang, Rowley,...

In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy import linalg

In [2]:
def fGEDMD(X, Y, tol=1e-12, type: str=None, k=-1, weights=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    if(weights is not None):
        X = X*weights; Y=Y*weights
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    A = Y @ (V/Sigma) @ np.conjugate(U.T)
    """S_k = np.conjugate(U.T)@B_k ###nema potrebe za ovim! - direktno eig..
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W"""
    if(type=="exact"):
        return "nema jos exact implementacije..."
        Z = B_k@W 
    Lambda, Z = np.linalg.eig(A)
    #print(A @ Z - ((np.multiply(Z, Lambda))))
    r = np.linalg.norm(A @ Z - np.multiply(Z, Lambda), axis=0)
    
    return Z, Lambda, r

In [3]:
def fGEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    m = X.shape[0]
    M = np.bmat([X.T, Y.T])
    Q, L = np.linalg.qr(M)
    #print(Q[:,:m].T@Q[:,:m]) #provjera ortogonalnosti
    L = L.T
    Z, Lambda, r = fGEDMD(L[:m,:m], L[m:,:m], tol, type)
    return Z, Lambda, r, Q, L 

In [4]:
def givens_for_adding_fdmd(R):
    M = R.shape[0]
    Giv = np.eye(M, M)
    for i in range(R.shape[1]):
        x1 = R[i,i]
        xz = R[M-1,i]
        korijen = np.sqrt(x1**2+xz**2)
        csn=x1/korijen; sn=-xz/korijen
        G_m_kutevi = np.array([[csn, -sn], [sn, csn]])
        Giv[[i,M-1], :] = (G_m_kutevi@Giv[[i,M-1], :])
        R[[i,M-1],i:] = (G_m_kutevi@R[[i,M-1],i:]) 
    #provjereno R dobar i Giv ortogonalan.
    return Giv.T[:,:-1], R[:-1,:]

In [5]:
def fDMD_added_snap(v, Q, L, tol=1e-12, type: str=None, k=-1, weights=None, ngram=5):
    #v tipa 1 x 2m => prvih m je zadnji element u Y, drugih m je novi element
    # ngram - treba li tu reortogonalizacija? mislim da NE
    m=Q.shape[1]//2; n=Q.shape[0]
    R = np.bmat([[L.T],[v.T]])
    #U, R = np.linalg.qr(R)
    U, R = givens_for_adding_fdmd(R)
    R=R.T #novi L - sad je R donjetrokutasta
    Q = np.vstack((Q@U[:2*m,:],U[-1,:].reshape(1,-1)))
    #novi Z i Lambda i r
    Z, Lambda, r = fGEDMD(R[:m,:m], R[m:,:m], tol, type)
    return Z, Lambda, r, Q, R

In [ ]:
#radi dobro, ali samo za micanje 1 po 1
### treba dodati mogucnost vise reortogonalizacije!!! kada m skoro isti kao n, jako brzo izgubi ortg.
def givens_for_removing(Q, R, k_reortg=2):
    q_van = Q[0,:].reshape(-1,1)

    #tu radim vektor e1 - ovdje bas kanonski
    e1 = np.zeros(Q.shape[0]); e1[0]=1.0; e1=e1.reshape(-1,1)
    ##############################################################################
    # # #       CGS
    ##############################################################################
    v1 = e1-(Q@(Q.T@e1)) #mogu li umjesto Q.T@e1 odmah napisati q_van? NE

    # # #       MGS
    #################   PUNO BOLJE!! Cak i kada 42x40 ili 22x20 matrica!!
    e1_proj = e1
    for i in range(Q.shape[0]):
        e1_proj = e1_proj - Q[:,i:i+1]@(Q[:,i:i+1].T@e1_proj)

    v1 = e1_proj

    ###reortogonalizacija
    r1=0
    for k in range(k_reortg):  #ovdje isto moze MGS
        s1 = Q.T@v1
        #r1 = r1+s1  ##ovo je za onu matricu I r \\ 0 \rho , ali ona mi ne treba, nju trokutasta "pojede"
        u1 = Q@s1
        v1=v1-u1

    rho = np.linalg.norm(v1) #mogao bi biti i - np.linalg.norm(v1)..
    #print("rho",rho)
    q_novi = v1/rho #print(q_novi) #ovo je novi q, na prvom mjestu ce biti tocno rho (tj ortogonaliziran zadnji dodani stupac e1..)
    #print(np.vstack((rho, q_novi))) #NE, q_novi ima na prvom mjestu rho!
    

    #je li ortogonalno?  JE
    #print("je li ortg novi stupac", np.linalg.norm(Q.T@q_novi))

    Q_prosireno = np.hstack((Q,q_novi))

    #print("norma novog:",np.linalg.norm(q_novi))

    #############################################################################
    #ovaj gornji dio mi se čini ok, ovaj donji treba urediti, greska sa 10^-15 na gornjem dijelu padne na 10^-14 nakon sto se izvrsi donji..

    ######################################################################

    #print("q_p",np.allclose(Q_prosireno.T@Q_prosireno, np.eye(Q_prosireno.shape[1])))
    R_prosireno = np.vstack((R, np.zeros((1,R.shape[1]))))

    len_Q = Q.shape[1]
    for i in range(len_Q-1, -1, -1):
        #print(i)
        x1 = q_van[i,0]; #ok - uzela sam transponirano.. 
        xz = Q_prosireno[0, len_Q]
        if(np.absolute(x1) > np.absolute(xz)):
            mmm = np.absolute(x1)
            korijen = mmm*np.sqrt(1 + (xz/mmm)*(xz/mmm))
        else:
            mmm = np.absolute(xz)
            korijen = mmm*np.sqrt(1 + (x1/mmm)*(x1/mmm))
        #korijen = np.sqrt(x1**2 + xz**2) #jednostavniji nacin, ali numericki losiji
        cs = xz/korijen; sn = x1/korijen #veca greska ako stavim - uz sin!
        #Q_prosireno[:,[i, len_Q]] = Q_prosireno[:,[i, len_Q]]@np.array([[cs, sn], [-sn, cs]]) 
        
        ##verzija po onom radu - isto nakon nekog vremena gubim ortogonalnost!
        Q_prosireno[0,i] = 0; Q_prosireno[0,len_Q] = korijen

        for j in range(1, Q.shape[0]):
            q_l = Q_prosireno[j, i]; q_z = Q_prosireno[j, len_Q]
            Q_prosireno[j,len_Q] = sn*q_l+cs*q_z
            Q_prosireno[j, i] = (cs*q_z+sn*q_l+q_l)*cs/(1+sn)-q_z
        #
        # print(np.array([[cs, sn], [-sn, cs]]).T@np.array([[cs, sn], [-sn, cs]]))

        #Q_prosireno[1:,[i, len_Q]] = ((Q_prosireno[1:,[i,len_Q]])@np.array([[cs, sn], [-sn, cs]]))
        #print("Q_pros ortg?",np.allclose(Q_prosireno.T@Q_prosireno, np.eye(Q_prosireno.shape[1]))) #u jednom trenu nakon puno iteracija gubim ortogonalnost

        #na R radim isto to slijeva.
        R_prosireno[[i,len_Q],:] = np.array([[cs, -sn], [sn, cs]])@R_prosireno[[i, len_Q], :]
        #print(R_prosireno)
    #print(R_prosireno)
    #print(Q_prosireno[0,:])
    return Q_prosireno[1:,:-1], R_prosireno[:-1,:]
    

In [ ]:
#radi dobro, ali samo za micanje 1 po 1
### treba dodati mogucnost vise reortogonalizacije!!! kada m skoro isti kao n, jako brzo izgubi ortg.
def random_givens_for_removing(Q, R):
    q_van = Q[0,:].reshape(-1,1)

    #tu radim vektor e1 - ovdje bas kanonski
    #e1 = np.zeros(Q.shape[0]); e1[0]=1.0; e1=e1.reshape(-1,1)
    #random verzija
    ### FUNKCIONIRA ZA ORTG ALI NIJE DOBRO ZA R... 
    e1=np.random.normal(size=Q.shape[0]); e1=e1.reshape(-1,1) #isti rezultati.. FUNKCIONIRA KADA dobri odnosi m i n (kada su širi:)) za ortgonalnost, za R NE!!
    v1 = e1-(Q@(Q.T@e1)) #mogu li umjesto Q.T@e1 odmah napisati q_van? NE
    
    rho = np.linalg.norm(v1) #mogao bi biti i - np.linalg.norm(v1)..
    print("rho",rho)
    while(rho<2):
        #print("v1:", v1.T) 
        #ako preblizu prostoru koji vec imam, uzmi novi random vektor
        e1=np.random.normal(size=Q.shape[0]); e1=e1.reshape(-1,1) 
        v1 = e1-(Q@(Q.T@e1)) #mogu li umjesto Q.T@e1 odmah napisati q_van? NE
        rho = np.linalg.norm(v1) 
        print("rho", rho)
    q_novi = v1/rho #print(q_novi) #ovo je novi q, na prvom mjestu ce biti tocno rho (tj ortogonaliziran zadnji dodani stupac e1..)
    #print(np.vstack((rho, q_novi))) #NE, q_novi ima na prvom mjestu rho!

    #je li ortogonalno?  JE
    #print("je li ortg novi stupac", np.linalg.norm(Q.T@q_novi))

    Q_prosireno = np.hstack((Q,q_novi))

    #print("norma novog:",np.linalg.norm(q_novi))

    #############################################################################
    #ovaj gornji dio mi se čini ok, ovaj donji treba urediti, greska sa 10^-15 na gornjem dijelu padne na 10^-14 nakon sto se izvrsi donji..

    ######################################################################

    #print("q_p",np.allclose(Q_prosireno.T@Q_prosireno, np.eye(Q_prosireno.shape[1])))
    R_prosireno = np.vstack((R, np.zeros((1,R.shape[1]))))

    len_Q = Q.shape[1]
    for i in range(len_Q-1, -1, -1):
        #print(i)
        x1 = q_van[i,0]; #ok - uzela sam transponirano.. 
        xz = Q_prosireno[0, len_Q]
        if(np.absolute(x1) > np.absolute(xz)):
            mmm = np.absolute(x1)
            korijen = mmm*np.sqrt(1 + (xz/mmm)*(xz/mmm))
        else:
            mmm = np.absolute(xz)
            korijen = mmm*np.sqrt(1 + (x1/mmm)*(x1/mmm))
        #korijen = np.sqrt(x1**2 + xz**2) #jednostavniji nacin, ali numericki losiji
        cs = xz/korijen; sn = x1/korijen #veca greska ako stavim - uz sin!
        #Q_prosireno[:,[i, len_Q]] = Q_prosireno[:,[i, len_Q]]@np.array([[cs, sn], [-sn, cs]]) 
        
        ##verzija po onom radu - isto nakon nekog vremena gubim ortogonalnost!
        Q_prosireno[0,i] = 0; Q_prosireno[0,len_Q] = korijen

        for j in range(1, Q.shape[0]):
            q_l = Q_prosireno[j, i]; q_z = Q_prosireno[j, len_Q]
            Q_prosireno[j,len_Q] = sn*q_l+cs*q_z
            Q_prosireno[j, i] = (cs*q_z+sn*q_l+q_l)*cs/(1+sn)-q_z
        #
        # print(np.array([[cs, sn], [-sn, cs]]).T@np.array([[cs, sn], [-sn, cs]]))

        #Q_prosireno[1:,[i, len_Q]] = ((Q_prosireno[1:,[i,len_Q]])@np.array([[cs, sn], [-sn, cs]]))
        #print("Q_pros ortg?",np.allclose(Q_prosireno.T@Q_prosireno, np.eye(Q_prosireno.shape[1]))) #u jednom trenu nakon puno iteracija gubim ortogonalnost

        #na R radim isto to slijeva.
        R_prosireno[[i,len_Q],:] = np.array([[cs, -sn], [sn, cs]])@R_prosireno[[i, len_Q], :]
        #print(R_prosireno)
    if(rho<0.1):
        print(Q_prosireno[:,-1]) #zadnji stupac, trebao bi biti \pm 1,0,0,...,0
    print(R_prosireno)
    print(Q_prosireno[0,:])
    return Q_prosireno[1:,:-1], R_prosireno[:-1,:]
    

In [8]:
###ovo funkcionira samo kada Q kvadratna ortogonalna!!!!!!!!!!!!!!!!!!!!!
## iz rada Hammerling, Lucas
def givens_for_removing2(Q, R):
    
    len_Q = Q.shape[1]
    for i in range(len_Q-1, -1, -1):
        #print(i)
        x1 = Q[0, 0]; #ok - uzela sam transponirano.. 
        xz = Q[0, i]
        if(np.absolute(x1) > np.absolute(xz)):
            mmm = np.absolute(x1)
            korijen = mmm*np.sqrt(1 + (xz/mmm)*(xz/mmm))
        else:
            mmm = np.absolute(xz)
            korijen = mmm*np.sqrt(1 + (x1/mmm)*(x1/mmm))
        #korijen = np.sqrt(x1**2 + xz**2) #jednostavniji nacin, ali numericki losiji
        cs = x1/korijen; sn = -xz/korijen 
        Q[:,[0, i]] = Q[:,[0, i]]@np.array([[cs, -sn], [sn, cs]]) 

        ##verzija po onom radu - isto nakon nekog vremena gubim ortogonalnost!
        """for j in range(Q.shape[0]):
            q_l = Q_prosireno[j, i]; q_z = Q_prosireno[j, len_Q]
            Q_prosireno[j,len_Q] = sn*q_l+cs*q_z
            Q_prosireno[j, i] = (cs*q_z+sn*q_l+q_l)*cs/(1+sn)-q_z"""
        #

        #na R radim isto to slijeva.
        R[[0,i],:] = np.array([[cs, sn], [-sn, cs]])@R[[0, i], :]
    print(Q[:,0]) #nisu 0!!!!!
    return Q,R
    

In [9]:
def fDMD_discarding_snap(Q, L, num_out=1, tol=1e-12, type: str=None):
    m=Q.shape[1]//2
    #U, P = np.linalg.qr(Q[num_out:,:])
    Q, L = givens_for_removing(Q, L.T, 2) #tu treba dodati reortg
    L = L.T
    Z, Lambda, r = fGEDMD(L[:m,:m], L[m:,:m], tol, type)
    return Z, Lambda, r, Q, L

In [ ]:
### Ovo bi trebalo biti isto?

def DMD_alpha_for_reconstruction(X, Z, indices, L, weights=None):
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S
    Z = modes,
    indices = indices which we want to work with - from 1/r graph
    weights = np.array tip ili lista"""
    #treba li se formirati Vandermondeova matrica?
    if (indices == 'all'):
        indices=np.array([i for i in range (Z.shape[1])])
    m=X.shape[1]; l=indices.shape[0]
    Z = Z[:,indices]
    Q, R = np.linalg.qr(Z) # Q je tipa duljina_snapshota(N) x l, R je tipa lxl
    if(weights is None):
        weights=np.ones((m)).reshape(-1)
    weights=np.array(weights)
    pom=np.vander(L[indices], m, increasing=True)*weights
    alpha= np.multiply(np.conj(R.T)@R, np.conj(pom @ np.conj(pom.T)))
    G = (np.conj(Q.T) @ X)[:l, :]  
    alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G))@np.ones((m, 1)), assume_a='pos')
    #alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G*weights))@np.ones((m, 1)), assume_a='pos')
    return Z, L[indices], alpha.reshape(-1)

def DMD_reconstruction(X, Z, indices, L, times, weights=None, real=True): #mozda da prima vektor napraviti..
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S,
    Z = dmd modes (returned from some version of DMD, ex. GEDMDQ)
    L = dmd eigs (returned from some version of DMD, ex. GEDMDQ)
    time = integer, which datasnapshot you want to reconstruct/predict"""
    Z_l, L, alpha = DMD_alpha_for_reconstruction(X, Z, indices, L, weights)
    num=np.asarray(times).shape[0]
    recs = np.empty((Z_l.shape[0], num), dtype=complex)
    for i in range(num):
        recs[:,i] = Z_l@(L**(times[i])*alpha) #mislim da je times[i] jer ovdje krecemo od 0, za razliku od matlaba gdje krecemo od 1
    if real:
        return np.real(recs) #ako sve realno, ovo ce biti realno za svaki i, samo ce zapis biti u obliku kompleksnog; zato saljemo np.real(recs)!
    else:
        return recs